In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from loguru import logger
import yaml
import sys
from pyspark.sql import SparkSession
import pandas as pd

from mlops_course.config import ProjectConfig
from mlops_course.data_processor import DataProcessor
from marvelous.logging import setup_logging
from marvelous.timer import Timer

config = ProjectConfig.from_yaml(config_path="../project_config.yml", env="dev")

setup_logging(log_file="logs/marvelous-1.log")

logger.info("Configuration loaded:")
logger.info(yaml.dump(config, default_flow_style=False))

2025-05-10 09:40:03 | INFO | 2428171464:<module> - Configuration loaded:
2025-05-10 09:40:03 | INFO | 2428171464:<module> - !!python/object:mlops_course.config.ProjectConfig
__dict__:
  cat_features:
  - name
  - full_name
  - birth_date
  - positions
  - nationality
  - preferred_foot
  - body_type
  - national_team
  - national_team_position
  catalog_name: mlops_dev
  num_features:
  - age
  - height_cm
  - weight_kgs
  - overall_rating
  - potential
  - wage_euro
  - international_reputation(1-5)
  - weak_foot(1-5)
  - skill_moves(1-5)
  - release_clause_euro
  - national_rating
  - national_jersey_number
  - crossing
  - finishing
  - heading_accuracy
  - short_passing
  - volleys
  - dribbling
  - curve
  - freekick_accuracy
  - long_passing
  - ball_control
  - acceleration
  - sprint_speed
  - agility
  - reactions
  - balance
  - shot_power
  - jumping
  - stamina
  - strength
  - long_shots
  - aggression
  - interceptions
  - positioning
  - vision
  - penalties
  - composur

In [10]:
# Load the fifa players dataset
spark = SparkSession.builder.getOrCreate()

filepath = "../data/fifa_players.csv"

# Load the data
df = pd.read_csv(filepath)


In [11]:
# Load the fifa players dataset
with Timer() as preprocess_timer:
    # Initialize DataProcessor
    data_processor = DataProcessor(df, config, spark)

    # Preprocess the data
    data_processor.preprocess()

logger.info(f"Data preprocessing: {preprocess_timer}")

KeyError: 'international_reputation(1-5)'

In [ ]:
# Split the data
X_train, X_test = data_processor.split_data()
logger.info("Training set shape: %s", X_train.shape)
logger.info("Test set shape: %s", X_test.shape)

2025-05-10 09:33:48 | INFO | 2594902236:<module> - Training set shape: %s
2025-05-10 09:33:48 | INFO | 2594902236:<module> - Test set shape: %s


In [ ]:
# Save to catalog
logger.info("Saving data to catalog")
data_processor.save_to_catalog(X_train, X_test)

# Enable change data feed (only once!)
logger.info("Enable change data feed")
data_processor.enable_change_data_feed()

2025-05-10 09:34:00 | INFO | 737852268:<module> - Saving data to catalog


AnalysisException: [DELTA_INVALID_CHARACTERS_IN_COLUMN_NAMES] Found invalid character(s) among ' ,;{}()\n\t=' in the column names of your schema.
Invalid column names: international_reputation(1-5).
Please use other characters and try again.
Alternatively, enable Column Mapping to keep using these characters.

JVM stacktrace:
com.databricks.sql.transaction.tahoe.DeltaAnalysisException
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.foundInvalidCharsInColumnNames(DeltaErrors.scala:2210)
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.foundInvalidCharsInColumnNames$(DeltaErrors.scala:2209)
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.foundInvalidCharsInColumnNames(DeltaErrors.scala:3579)
	at com.databricks.sql.transaction.tahoe.schema.SchemaUtils$.checkSchemaFieldNames(SchemaUtils.scala:1290)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.assertMetadata(OptimisticTransaction.scala:791)
	at com.databricks.sql.transaction.tahoe.OptimisticTrans